In [ ]:
# import libraries
import numpy as np
import pandas as pd
from IPython.display import display
from IPython.core.display import HTML
from sklearn.model_selection import train_test_split
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error, precision_score, recall_score, f1_score
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm

In [3]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split

In [5]:
df_books = pd.read_csv("cleaned_books.csv")
df_ratings = pd.read_csv("cleaned_ratings.csv")

In [19]:
# limit the ratings per book to max 1000
df_ratings = df_ratings.groupby('user_id').filter(lambda x: len(x) >= 5)
df_ratings = df_ratings[df_ratings["book_id"].isin(df_books.index)]
df_ratings = (
    df_ratings.groupby("book_id")
    .apply(lambda x: x.sample(min(1000, len(x))))
    .reset_index(drop=True)
)

In [20]:
len(df_ratings)  

3457285

In [21]:
# Map IDs to indices
user_map = {u: i for i, u in enumerate(df_ratings['user_id'].unique())}
item_map = {b: i for i, b in enumerate(df_ratings['book_id'].unique())}
df_ratings['user_idx'] = df_ratings['user_id'].map(user_map)
df_ratings['item_idx'] = df_ratings['book_id'].map(item_map)

n_users = len(user_map)
n_items = len(item_map)

In [22]:
train_df, test_df = train_test_split(df_ratings, test_size=0.2, random_state=42)

In [23]:
R_train = np.zeros((n_users, n_items))

for row in train_df.itertuples():
    R_train[row.user_idx, row.item_idx] = row.rating
    
# SCompute user mean ratings (avoid divide by zero)
user_means = np.true_divide(R_train.sum(1), (R_train != 0).sum(1))
user_means = np.nan_to_num(user_means)  # convert NaNs to 0 if needed

# Center the ratings (subtract user mean from each rating)
R_centered = R_train - user_means[:, np.newaxis]

In [31]:
# Number of latent factors
k = 20  
U, sigma, Vt = svds(R_centered, k=k)
sigma = np.diag(sigma)

predicted_ratings_matrix = np.dot(np.dot(U, sigma), Vt)
predicted_ratings_matrix += user_means[:, np.newaxis]

In [9]:
def compute_rmse(pred_matrix, test_df):
    preds = []
    truths = []

    for row in test_df.itertuples():
        user_idx = row.user_idx
        item_idx = row.item_idx
        pred_rating = pred_matrix[user_idx, item_idx]
        preds.append(pred_rating)
        truths.append(row.rating)

    rmse = np.sqrt(mean_squared_error(truths, preds))
    return round(rmse, 4)

In [10]:
def compute_mae(pred_matrix, test_df):
    preds = []
    truths = []

    for row in test_df.itertuples():
        user_idx = row.user_idx
        item_idx = row.item_idx
        pred_rating = pred_matrix[user_idx, item_idx]
        preds.append(pred_rating)
        truths.append(row.rating)

    mae = mean_absolute_error(truths, preds)
    return round(mae, 4)

In [11]:
def apk(actual, predicted, k=5):
    if not actual:
        return 0.0
    predicted = predicted[:k]

    score = 0.0
    hits = 0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            hits += 1
            score += hits / (i + 1)
    return score / min(len(actual), k)

def reciprocal_rank(actual, predicted):
    for i, p in enumerate(predicted):
        if p in actual:
            return 1.0 / (i + 1)
    return 0.0

In [12]:
def pre_recall_f1_map_mrr(pred_matrix, train_df, test_df, k=10, threshold=4.0):
    correct = 0
    total = 0
    total_score = 0
    total_rr = 0
    total_relevant = 0
    user_count = 0
    train_user_items = train_df.groupby("user_idx")["item_idx"].apply(set)

    for user in tqdm(test_df['user_idx'].unique(), desc="Processing evaluation (Precision, Recall, F1, MRR, MAP)", unit="user"):
        rated_items = train_user_items.get(user, set())
        user_pred = pred_matrix[user]
        
        # Exclude items seen in training
        unseen_items = np.setdiff1d(np.arange(pred_matrix.shape[1]), list(rated_items))
        top_k_items = unseen_items[np.argsort(user_pred[unseen_items])[::-1][:k]]

        # True relevant items in test set for this user
        user_test = test_df[test_df['user_idx'] == user]
        relevant_items = user_test[user_test['rating'] >= threshold]['item_idx'].values
        
        # Calculate precision
        correct += len(set(top_k_items) & set(relevant_items))
        total += k
        total_relevant += len(relevant_items)
        
        actual_items = test_df[(test_df['user_idx'] == user) & (test_df['rating'] >= threshold)]['item_idx'].tolist()
        #Calculate MAP and MRR
        if actual_items:
            total_score += apk(actual_items, list(top_k_items), k)
            total_rr += reciprocal_rank(actual_items, list(top_k_items))
            user_count += 1      

    precision = round(correct / total, 4)
    recall = round(correct / total_relevant, 4) if total_relevant else 0
    f1 = round(2 * precision * recall / (precision + recall), 4) if (precision + recall) > 0 else 0
    map = round(total_score / user_count, 4) if user_count else 0
    mrr = round(total_rr / user_count, 4) if user_count else 0
    return precision, recall, f1, map, mrr


In [25]:
print("RMSE:", compute_rmse(predicted_ratings_matrix, test_df))
print("MAE:", compute_mae(predicted_ratings_matrix, test_df))

precision, recall, f1, map, mrr = pre_recall_f1_map_mrr(predicted_ratings_matrix, train_df, test_df, k=5, threshold=4.0)
print("Precision@5:", precision)
print("Recall@5:", recall)
print("F1@5:", f1)
print("MAP@5:", map)
print("MRR@5:", mrr)

RMSE: 3.9117
MAE: 3.7898


Processing evaluation (Precision, Recall, F1, MRR, MAP): 100%|██████████| 53402/53402 [11:14<00:00, 79.15user/s] 


Precision@5: 0.0296
Recall@5: 0.0167
F1@5: 0.0214
MAP@5: 0.0198
MRR@5: 0.0601


In [26]:
def recommend_books(user_id, R_train, predicted_ratings, n=5):
    user_idx = user_map[user_id]
    user_ratings = R_train[user_idx]
    preds = predicted_ratings[user_idx]
    
    # Books not rated by user
    unrated_indices = np.where(user_ratings == 0)[0]
    recommended_indices = unrated_indices[np.argsort(preds[unrated_indices])[::-1][:n]]

    # Map back to book IDs
    item_map_rev = {i: b for b, i in item_map.items()}
    recommended_books = [(item_map_rev[i], preds[i]) for i in recommended_indices]
    return recommended_books

In [33]:
# Function to display images in a DataFrame
def display_images(df, image_column):
    # Create an HTML representation of the DataFrame with images
    html = df.to_html(escape=False, formatters={
        image_column: lambda url: f'<img src="{url}" width="100">'
    })
    display(HTML(html))

In [32]:
rcmd = recommend_books(123, R_train, predicted_ratings_matrix)
rcmd_df = pd.DataFrame(rcmd, columns=["book_id", "predicted_rating"])
rcmd_df = rcmd_df.merge(df_books[['book_id', 'title', 'image_url']], on='book_id', how='left')
rcmd_df['predicted_rating'] = rcmd_df['predicted_rating'].round(4)
display_images(rcmd_df, 'image_url')

,book_id,predicted_rating,title,image_url
0,1052,0.3383,Handle with Care,
1,1101,0.3370,Change of Heart,
2,552,0.3208,The Rescue,
3,1107,0.3194,Little Earthquakes,
4,952,0.3181,"Shopaholic & Baby (Shopaholic, #5)",


In [28]:
user_id = 123

# Filter ratings for user 123
user_ratings = df_ratings[df_ratings['user_id'] == user_id]

# Sort by rating in descending order and get the top 5
top_5_actual_ratings = user_ratings.sort_values(by='rating', ascending=False).head(5)

# Merge with book details to display titles
top_5_actual_ratings = top_5_actual_ratings.merge(df_books[['book_id', 'title', 'image_url']], on='book_id', how='left')

# Display the result
display_images(top_5_actual_ratings, 'image_url')

,user_id,book_id,rating,user_idx,item_idx,title,image_url
0,123,26,5,18464,24,"The Da Vinci Code (Robert Langdon, #2)",
1,123,2302,5,18464,2193,Scarlett,
2,123,1481,5,18464,1418,Her Fearful Symmetry,
3,123,1482,5,18464,1419,The Boston Girl,
4,123,1644,5,18464,1574,Peace Like a River,


In [6]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_ratings[['user_id', 'book_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)


In [7]:
from surprise import SVD
from surprise import accuracy

model = SVD(n_factors=20)  # like latent factor count
model.fit(trainset)

In [40]:
predictions = model.test(testset)

In [38]:
from collections import defaultdict

def get_top_n_df(predictions, n=5, user_id=None):
    top_n = defaultdict(list)
    
    # Group predictions by user
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est, true_r))
    
    # Sort and retain top-n
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    
    # Filter if user_id is provided
    if user_id is not None:
        top_n = {user_id: top_n.get(user_id, [])}
    
    # Convert to DataFrame
    rows = []
    for uid, user_ratings in top_n.items():
        for iid, est_rating, true_rating in user_ratings:
            rows.append((iid, est_rating, true_rating))
    
    df_top_n = pd.DataFrame(rows, columns=['book_id', 'predicted_rating', 'actual_rating'])
    return df_top_n


In [39]:
uid=2056
df_all = get_top_n_df(predictions, n=5)
df_user = get_top_n_df(predictions, n=5, user_id=uid)
df_user = df_user.merge(df_books[['book_id', 'title', 'image_url']], on='book_id', how='left')

print("Top 5 recommended books for user", uid,":")
display_images(df_user, 'image_url')


Top 5 recommended books for user 2056 :


,book_id,predicted_rating,actual_rating,title,image_url
0,1627,4.463002,4.0,"Twelve Sharp (Stephanie Plum, #12)",
1,1606,4.448722,5.0,"Ten Big Ones (Stephanie Plum, #10)",
2,743,4.271887,3.0,"Lamb: The Gospel According to Biff, Christ's Childhood Pal",
3,2138,4.239610,3.0,"Sizzling Sixteen (Stephanie Plum, #16)",
4,207,4.166588,5.0,"One for the Money (Stephanie Plum, #1)",


In [46]:
def metrics_at_k(predictions, k=5, threshold=4.0):
    user_est_true = defaultdict(list)

    for uid, iid, true_r, est, _ in predictions:
        user_est_true[uid].append((iid, est, true_r))

    precisions, recalls, f1s = [], [], []
    average_precisions, reciprocal_ranks = [], []

    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_k = user_ratings[:k]

        rel_items = [1 if true_r >= threshold else 0 for (_, _, true_r) in user_ratings]
        top_k_rel = [1 if true_r >= threshold else 0 for (_, _, true_r) in top_k]

        # Precision, Recall
        n_rel = sum(rel_items)
        n_rel_k = sum(top_k_rel)

        precision = n_rel_k / k if k else 0
        recall = n_rel_k / n_rel if n_rel else 0
        precisions.append(precision)
        recalls.append(recall)

        # F1@K
        if precision + recall > 0:
            f1 = 2 * (precision * recall) / (precision + recall)
        else:
            f1 = 0
        f1s.append(f1)

        # MAP@K
        num_hits, sum_prec = 0, 0.0
        for i, rel in enumerate(top_k_rel, 1):
            if rel:
                num_hits += 1
                sum_prec += num_hits / i
        average_precisions.append(sum_prec / min(n_rel, k) if n_rel else 0)

        # MRR@K
        rr = 0
        for i, rel in enumerate(top_k_rel, 1):
            if rel:
                rr = 1 / i
                break
        reciprocal_ranks.append(rr)
        
    precision = np.mean(precisions)
    recall = np.mean(recalls)
    f1 = np.mean(f1s)
    map = np.mean(average_precisions)
    mrr = np.mean(reciprocal_ranks)

    return precision, recall, f1, map, mrr


In [42]:
def get_user_ranks(preds, threshold=4.0):
    user_est_true = defaultdict(list)
    for uid, iid, true_r, est, _ in preds:
        user_est_true[uid].append((est, true_r))
    
    APs = []
    RRs = []
    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        num_relevant = 0
        precision_sum = 0
        rr = 0
        for i, (est, true_r) in enumerate(user_ratings):
            relevant = true_r >= threshold
            if relevant:
                num_relevant += 1
                precision_sum += num_relevant / (i + 1)
                if rr == 0:
                    rr = 1 / (i + 1)
        if num_relevant > 0:
            APs.append(precision_sum / num_relevant)
            RRs.append(rr)
    
    map_score = np.mean(APs)
    mrr_score = np.mean(RRs)
    return map_score, mrr_score

In [47]:
rmse = accuracy.rmse(predictions, verbose=False)
mae = accuracy.mae(predictions, verbose=False)

threshold = 4.0
y_true = [int(true_r >= threshold) for (_, _, true_r, _, _) in predictions]
y_pred = [int(est >= threshold) for (_, _, _, est, _) in predictions]

precision, recall, f1, map_score, mrr_score = metrics_at_k(predictions, k=5, threshold=threshold)

print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

print(f"Precision@5: {precision:.4f}")
print(f"Recall@5: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"MAP: {map_score:.4f}")
print(f"MRR: {mrr_score:.4f}")


RMSE: 0.8327
MAE: 0.6461
Precision@5: 0.8246
Recall@5: 0.3229
F1 Score: 0.4414
MAP: 0.7769
MRR: 0.9283
